In [1]:
## ABRIR DATASET ACTUALIZADO
import pickle

with open("dataset_actualizado.pkl", "rb") as f:
    datos = pickle.load(f)

#print(datos);print(datos.keys());print(datos["data"]);print(datos["distancias"]);print(datos["tiempos"]);print(datos["costes"])


In [7]:
## CREAR EL CONJUNTO DE NODOS ##
base_operativa_ficticia=-1
N = [i for i,d in enumerate(datos["data"]) if d['nodo']=="carga"]
F = [i for i,d in enumerate(datos["data"]) if d['nodo']=="cargador"]
V = list(range(len(datos["data"])))+[base_operativa_ficticia]
base_operativa = [i for i,d in enumerate(datos["data"]) if d['nodo']=="parking"][0]
descarga = [i for i,d in enumerate(datos["data"]) if d['nodo']=="descarga"][0]

print(V);print(N);print(F);print(base_operativa);print(descarga);print(base_operativa_ficticia)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1]
[1, 2, 3, 4]
[6, 7, 8, 9, 10]
0
5
-1


In [3]:
# M-GRANDE 
M = 1000

In [4]:
# CREACIÓN DE ARRAYS
import numpy as np
c_0 = np.array(datos["costes"])
w_0 = np.array([d["w"] for d in datos["data"]])
tiempos_0 = np.array(datos["tiempos"])

In [5]:
# AÑADIR EL INDICE -1 (BASE OPERATIVA FICTICIA)

#c
fila_nueva_c = c_0[0, :].reshape(1, -1)
c = np.vstack((c_0, fila_nueva_c))
columna_nueva_c = c[:, 0].reshape(-1, 1)
c = np.hstack((c, columna_nueva_c))

#w
w=np.append(w_0,0)

#tiempos
fila_nueva_t = tiempos_0[0, :].reshape(1, -1)
tiempos = np.vstack((tiempos_0, fila_nueva_t))
columna_nueva_t = tiempos[:, 0].reshape(-1, 1)
tiempos = np.hstack((tiempos, columna_nueva_t))


In [17]:
#tiempos[5,-1]
tiempos[0,1]+tiempos[1,6]+tiempos[6,2]


np.float64(59.30571428571429)

In [10]:
## PROGRAMA PRINCIPAL
import gurobipy as gp
from gurobipy import GRB

# Creamos el modelo
m = gp.Model("problema1")

# Creamos las variables
x = m.addVars(V,V, vtype=GRB.BINARY, name="x")
z = m.addVars(F, vtype=GRB.BINARY, name="z")
t = m.addVars(V, lb=0.0, ub= 15*60, vtype=GRB.CONTINUOUS, name="t")
m.update()

# Definimos la función objetivo
obj = gp.quicksum(c[i, j] * x[i, j] for i in V for j in V if i != j) + gp.quicksum(w[i] * z[i] for i in F)
m.setObjective(obj, GRB.MINIMIZE)

# Definimos las restricciones
# r0: Inicializamos el tiempo, base operativa = 0
m.addConstr(t[base_operativa] == 0, name="r0")
# r1
for i in V:
    if i not in F and i!= base_operativa_ficticia:
        m.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"r1_{i}")
# r2
for j in V:
    if j not in [base_operativa_ficticia, base_operativa]:
        m.addConstr(gp.quicksum(x[i,j] for i in V if i!=j)-gp.quicksum(x[j,h] for h in V if h!=j)==0, name=f"r2_{j}")
# r3
m.addConstr(gp.quicksum(x[base_operativa,j] for j in V if j not in [base_operativa, base_operativa_ficticia])==1, name="r3")
# r4
m.addConstr(gp.quicksum(x[i,base_operativa_ficticia] for i in V if i not in [base_operativa, base_operativa_ficticia])==1, name="r4")
# r5
m.addConstr(gp.quicksum(x[descarga, i] for i in N)==0, name="r5")
# r6
for j in V:
    m.addConstr(x[base_operativa_ficticia, j] == 0, name=f"r6_{j}")
# r7
for i in V:
    for j in V:
        if i!=j:
            m.addConstr(t[i]+tiempos[i,j]-t[j]<=M*(1-x[i,j]), name=f"r7_{i}_{j}")     
# r8
m.addConstr(gp.quicksum(x[i,j] for i in (N+[base_operativa]) for j in F if i!=j)==1, name="r8")
# r9
for i in F:
    for j in N:
        if i!=j:
            m.addConstr(x[i,j]<= z[i], name=f"r9_{i}_{j}")        
# r10
for i in N:
    for j in F:
        if i!=j:
            m.addConstr(x[i,j]<= z[j], name=f"r10_{i}_{j}") 

m.update()
m.optimize()
a= m.ObjVal
print("Funcion objetivo", a)
eps = 1e-6
for v in m.getVars():
    val = v.x
    if val is None:
        continue
    if abs(val) > eps:
        print(f"{v.VarName}={val:.2f}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1123)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 205 rows, 161 columns and 824 nonzeros
Model fingerprint: 0x56d8978e
Variable types: 12 continuous, 149 integer (149 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 1e+03]
Presolve removed 56 rows and 40 columns
Presolve time: 0.01s
Presolved: 149 rows, 121 columns, 1102 nonzeros
Variable types: 10 continuous, 111 integer (111 binary)
Found heuristic solution: objective 989.8520000

Root relaxation: objective 5.373415e+01, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   53.73415 

     0     0   61.97600    0   14  239.85200   61.97600  74.2%     -    0s
     0     0   61.97600    0   14  239.85200   61.97600  74.2%     -    0s
     0     0   61.98099    0   14  239.85200   61.98099  74.2%     -    0s
     0     0   61.98099    0   14  239.85200   61.98099  74.2%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     0   62.63200    0    8  239.85200   62.63200  73.9%     -    0s
     0     2   63.75800    0    8  239.85200   63.75800  73.4%     -    0s
*   36    11               6     235.8480000   64.13200  72.8%  10.7    1s
*   58     0               8     235.5280000  232.02400  1.49%  11.7    1s

Cutting planes:
  Learne